In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df.head()

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1722.0,32575.0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1688.0,32507.0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1696.0,32552.0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,0,0.0,0,0.8,0.0,1681.0,32423.0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,0,0.0,0,0.5,0.0,1704.0,32410.0,Prague_1_2020-12-09


In [ ]:
df = df[test.columns.tolist() + ['orders']]
df.head()


,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id,orders
0,Prague_1,2020-12-05,NaN,0,0,0,0,Prague_1_2020-12-05,6895.0
1,Prague_1,2020-12-06,NaN,0,0,0,0,Prague_1_2020-12-06,6584.0
2,Prague_1,2020-12-07,NaN,0,0,0,0,Prague_1_2020-12-07,7030.0
3,Prague_1,2020-12-08,NaN,0,0,0,0,Prague_1_2020-12-08,6550.0
4,Prague_1,2020-12-09,NaN,0,0,0,0,Prague_1_2020-12-09,6910.0


In [ ]:
df.shape

(7340, 9)

In [ ]:
df.isnull().sum()

warehouse                    0
date                         0
holiday_name              7122
holiday                      0
shops_closed                 0
winter_school_holidays       0
school_holidays              0
id                           0
orders                       0
dtype: int64

In [ ]:
df['holiday_name'].replace(np.nan, "No Holiday", inplace=True)
df.isnull().sum()

warehouse                 0
date                      0
holiday_name              0
holiday                   0
shops_closed              0
winter_school_holidays    0
school_holidays           0
id                        0
orders                    0
dtype: int64

In [ ]:
df.isna().sum()

warehouse                 0
date                      0
holiday_name              0
holiday                   0
shops_closed              0
winter_school_holidays    0
school_holidays           0
id                        0
orders                    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7340 entries, 0 to 7339
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   warehouse               7340 non-null   object 
 1   date                    7340 non-null   object 
 2   holiday_name            7340 non-null   object 
 3   holiday                 7340 non-null   int64  
 4   shops_closed            7340 non-null   int64  
 5   winter_school_holidays  7340 non-null   int64  
 6   school_holidays         7340 non-null   int64  
 7   id                      7340 non-null   object 
 8   orders                  7340 non-null   float64
dtypes: float64(1), int64(4), object(4)
memory usage: 516.2+ KB


In [ ]:
df.describe()

,holiday,shops_closed,winter_school_holidays,school_holidays,orders
count,7340.000000,7340.000000,7340.000000,7340.000000,7340.000000
mean,0.027248,0.010899,0.029973,0.007084,5535.443869
std,0.162816,0.103836,0.170524,0.083876,2182.680279
min,0.000000,0.000000,0.000000,0.000000,790.000000
25%,0.000000,0.000000,0.000000,0.000000,4434.000000
50%,0.000000,0.000000,0.000000,0.000000,5370.000000
75%,0.000000,0.000000,0.000000,0.000000,7009.250000
max,1.000000,1.000000,1.000000,1.000000,18139.000000


In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
X = df.drop('orders', axis=1)
y = df['orders']

In [ ]:
X['Year'] = X['date'].dt.year
X['Month'] = X['date'].dt.month
X['Day'] = X['date'].dt.day
X['day_of_the_week'] = X['date'].dt.dayofweek
X['is_weekend'] = (X['date'].dt.dayofweek >= 5).astype(int)
X['is_sunday'] = (X['date'].dt.dayofweek == 6).astype(int)
X['is_holiday'] = (X['holiday'] == 'Yes').astype(int)
X['dates_multiplied'] = X['Year'] * X['Month'] * X['Day']
X['quarter'] = X['date'].dt.quarter
X['day_of_year'] = X['date'].dt.dayofyear
X['week_of_year'] = X['date'].dt.isocalendar().week
X['month_sin'] = np.sin(2 * np.pi * X['Month'] / 12)
X['month_cos'] = np.cos(2 * np.pi * X['Month'] / 12)
X['is_month_end'] = X['date'].dt.is_month_end.astype(int)
X['hour'] = X['date'].dt.hour
X['minute'] = X['date'].dt.minute
X['is_morning'] = X['hour'].between(6, 12).astype(int)
X['is_afternoon'] = X['hour'].between(12, 18).astype(int)
X['is_evening'] = X['hour'].between(18, 24).astype(int)
X['is_night'] = X['hour'].between(0, 6).astype(int)
X['is_leap_year'] = X['date'].dt.is_leap_year.astype(int)
X['is_payday'] = X['date'].dt.day.isin([1, 15]).astype(int)
X['hour_sin'] = np.sin(2 * np.pi * X['hour'] / 24)
X['hour_cos'] = np.cos(2 * np.pi * X['hour'] / 24)


X[['city', 'number']] = X['warehouse'].str.split('_', expand=True)
X['number'] = X['number'].astype(int)
X.drop('warehouse', axis=1, inplace=True)




In [ ]:
X.columns

Index(['date', 'holiday_name', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'Year', 'Month', 'Day',
       'day_of_the_week', 'is_weekend', 'is_sunday', 'is_holiday',
       'dates_multiplied', 'quarter', 'day_of_year', 'week_of_year',
       'month_sin', 'month_cos', 'is_month_end', 'hour', 'minute',
       'is_morning', 'is_afternoon', 'is_evening', 'is_night', 'is_leap_year',
       'is_payday', 'hour_sin', 'hour_cos', 'city', 'number'],
      dtype='object')

In [ ]:
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
cat_cols, len(num_cols)

(Index(['holiday_name', 'city'], dtype='object'), 19)

In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_cols_encoded = pd.DataFrame(encoder.fit_transform(X[cat_cols]), columns=encoder.get_feature_names_out(cat_cols))
X_modified = pd.concat([X.drop(cat_cols, axis=1), cat_cols_encoded], axis=1)

In [ ]:
X_modified.drop('date', axis=1, inplace=True)

In [ ]:
l1 = Lasso(alpha=0.5)
l1.fit(X_modified, y)

before = len(X_modified.columns)
cols_reg = X_modified.columns[l1.coef_ != 0]
after = len(cols_reg)

X_modified = X_modified[cols_reg]

print(f"Dropped: {before - after} cols")

Dropped: 26 cols


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.218e+09, tolerance: 3.496e+06
  model = cd_fast.enet_coordinate_descent(


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_modified, y, test_size=0.2, random_state=42)

In [ ]:
lgbm = LGBMRegressor()
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred_lgbm)
print("Mean Absolute Percentage Error (MAPE):", mape)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001333 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 653
[LightGBM] [Info] Number of data points in the train set: 5872, number of used features: 22
[LightGBM] [Info] Start training from score 5512.430518
Mean Absolute Percentage Error (MAPE): 0.04079450396654915


In [ ]:
knn = KNeighborsRegressor(n_neighbors=70)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred_knn)
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Absolute Percentage Error (MAPE): 0.47991879480448896


In [ ]:
ex_gb = XGBRegressor(learning_rate=0.452)
ex_gb.fit(X_train, y_train)
y_pred_ex_gb = ex_gb.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred_ex_gb)
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Absolute Percentage Error (MAPE): 0.039458377607132965


In [ ]:
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=7, random_state=42)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred_gb)
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Absolute Percentage Error (MAPE): 0.03595041165620841


In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_forest = rf.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred_forest)
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Absolute Percentage Error (MAPE): 0.0460902105373775


In [ ]:
y_ensembled = (y_pred_ex_gb + y_pred_gb + y_pred_lgbm + y_pred_forest) / 4
mape = mean_absolute_percentage_error(y_test, y_ensembled)
print("Mean Absolute Percentage Error (MAPE):", mape)

Mean Absolute Percentage Error (MAPE): 0.03581599135719327


#**Test Now !!**

In [ ]:
test.shape

(397, 8)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   warehouse               397 non-null    object
 1   date                    397 non-null    object
 2   holiday_name            17 non-null     object
 3   holiday                 397 non-null    int64 
 4   shops_closed            397 non-null    int64 
 5   winter_school_holidays  397 non-null    int64 
 6   school_holidays         397 non-null    int64 
 7   id                      397 non-null    object
dtypes: int64(4), object(4)
memory usage: 24.9+ KB


In [ ]:
test['holiday_name'].replace(np.nan, "No Holiday", inplace=True)
test.isnull().sum()

warehouse                 0
date                      0
holiday_name              0
holiday                   0
shops_closed              0
winter_school_holidays    0
school_holidays           0
id                        0
dtype: int64

In [ ]:
test['date'] = pd.to_datetime(test['date'])

In [ ]:
final_test = test.drop('id', axis=1)

In [ ]:
final_test['Year'] = final_test['date'].dt.year
final_test['Month'] = final_test['date'].dt.month
final_test['Day'] = final_test['date'].dt.day
final_test['day_of_the_week'] = final_test['date'].dt.dayofweek
final_test['is_weekend'] = (final_test['date'].dt.dayofweek >= 5).astype(int)
final_test['is_sunday'] = (final_test['date'].dt.dayofweek == 6).astype(int)
final_test['is_holiday'] = (final_test['holiday'] == 'Yes').astype(int)
final_test['dates_multiplied'] = final_test['Year'] * final_test['Month'] * final_test['Day']
final_test['quarter'] = final_test['date'].dt.quarter
final_test['day_of_year'] = final_test['date'].dt.dayofyear
final_test['week_of_year'] = final_test['date'].dt.isocalendar().week
final_test['month_sin'] = np.sin(2 * np.pi * final_test['Month'] / 12)
final_test['month_cos'] = np.cos(2 * np.pi * final_test['Month'] / 12)
final_test['is_month_end'] = final_test['date'].dt.is_month_end.astype(int)
final_test['hour'] = final_test['date'].dt.hour
final_test['minute'] = final_test['date'].dt.minute
final_test['is_morning'] = final_test['hour'].between(6, 12).astype(int)
final_test['is_afternoon'] = final_test['hour'].between(12, 18).astype(int)
final_test['is_evening'] = final_test['hour'].between(18, 24).astype(int)
final_test['is_night'] = final_test['hour'].between(0, 6).astype(int)
final_test['is_leap_year'] = final_test['date'].dt.is_leap_year.astype(int)
final_test['is_payday'] = final_test['date'].dt.day.isin([1, 15]).astype(int)
X['hour_sin'] = np.sin(2 * np.pi * X['hour'] / 24)
X['hour_cos'] = np.cos(2 * np.pi * X['hour'] / 24)
final_test.drop('date', axis=1, inplace=True)





In [ ]:
#encoded_cols = pd.DataFrame(encoder.transform(final_test[cat_cols]), columns=encoder.get_feature_names_out(cat_cols))
#final_test = pd.concat([final_test.drop(cat_cols, axis=1), encoded_cols], axis=1)

In [ ]:
#final_test = final_test[cols_reg]

In [ ]:
"""
extreme_grad_pred = ex_gb.predict(final_test)
gb_pred = gb.predict(final_test)
lgbm_pred = lgbm.predict(final_test)
rf_pred = rf.predict(final_test)

final_ensembled = (extreme_grad_pred + gb_pred + lgbm_pred + rf_pred) / 4
final_df = pd.DataFrame({'id': test['id'], 'orders': final_ensembled})
final_df.head()
"""

"\nextreme_grad_pred = ex_gb.predict(final_test)\ngb_pred = gb.predict(final_test)\nlgbm_pred = lgbm.predict(final_test)\nrf_pred = rf.predict(final_test)\n\nfinal_ensembled = (extreme_grad_pred + gb_pred + lgbm_pred + rf_pred) / 4\nfinal_df = pd.DataFrame({'id': test['id'], 'orders': final_ensembled})\nfinal_df.head()\n"

In [ ]:
final_df.to_csv('Submission.csv', index=False)